Diagnoses use ICD9 - can cluster them into the chapters: https://en.wikipedia.org/wiki/List_of_ICD-9_codes

We already have:
- number of medications
- We have discharge dest and if they were admitted as an emergency


Initial thoughts:

- From the data, those with more medical history - more hospital visits, etc., more medications, 
- discharged to home also was less on non-readdmissions - but discharged to home /w some health service etc. was higher - aligns with people with more medical needs not doing welll after they leave hospital
- also those with "change" in medication see higher rates of readmission
- those with diabetes meds see higher rates of readmission

The story is, those with complex health needs, (made easier to get meds wrong if they've changed), are more likely to see readmission

NEED TO LOOK AT CHARLSON
https://github.com/samisaf/Calculating-Charlson-comorbidity-index-based-on-ICD9-codes/blob/master/input/CharlsonRules3.csv

In [1]:
import pandas as pd
import src.utils as utils

recalculate = False

In [2]:
if recalculate:
    file_path = "data/Diabetes.xlsx"

    # Load the Excel file into a pandas dataframe
    df = pd.read_excel(file_path)
    df['age_interval'] = df['age'].apply(utils.interval_type)
    df['age_factor'] = df['age_interval'].apply(utils.charlson_factor_age)
    df['diag_1_factor'] = df['diag_1'].apply(utils.charlson_factor_icd9)
    df['diag_2_factor'] = df['diag_2'].apply(utils.charlson_factor_icd9)
    df['diag_3_factor'] = df['diag_3'].apply(utils.charlson_factor_icd9)
    df['charlson_comorb_index'] = df.apply(lambda x: utils.charlson_comorb_index([x['diag_1'],x['diag_2'],x['diag_3']], x['age_interval']), axis=1)
    df.to_csv("data/Diabetes_Charlson.csv", index=False)
else:	
    df = pd.read_csv("data/Diabetes_Charlson.csv")

d:\PycharmProjects\readmission\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [10]:
df

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,...,readmitted,diag_1_desc,diag_2_desc,diag_3_desc,age_interval,age_factor,diag_1_factor,diag_2_factor,diag_3_factor,charlson_comorb_index
0,Caucasian,Female,[50-60),?,Elective,Discharged to home,Physician Referral,1,CP,Surgery-Neuro,...,False,Spinal stenosis in cervical region,Spinal stenosis in cervical region,"Effusion of joint, site unspecified","[50, 60)",1,0,0,0,1
1,Caucasian,Female,[20-30),[50-75),Urgent,Discharged to home,Physician Referral,2,UN,?,...,False,"First-degree perineal laceration, unspecified ...","Diabetes mellitus of mother, complicating preg...",Sideroblastic anemia,"[20, 30)",0,0,0,0,0
2,Caucasian,Male,[80-90),?,Not Available,Discharged/transferred to home with home healt...,NaN,7,MC,Family/GeneralPractice,...,True,Pneumococcal pneumonia [Streptococcus pneumoni...,"Congestive heart failure, unspecified",Hyperosmolality and/or hypernatremia,"[80, 90)",4,0,1,0,5
3,AfricanAmerican,Female,[50-60),?,Emergency,Discharged to home,Transfer from another health care facility,4,UN,?,...,False,Cellulitis and abscess of face,Streptococcus infection in conditions classifi...,Diabetes mellitus without mention of complicat...,"[50, 60)",1,0,0,1,2
4,AfricanAmerican,Female,[50-60),?,Emergency,Discharged to home,Emergency Room,5,?,Psychiatry,...,False,"Bipolar I disorder, single manic episode, unsp...",Diabetes mellitus without mention of complicat...,Depressive type psychosis,"[50, 60)",1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Caucasian,Female,[60-70),?,Elective,Discharged/transferred to home with home healt...,Physician Referral,3,MC,?,...,False,"Osteoarthrosis, generalized, site unspecified",Malignant essential hypertension,Pure hypercholesterolemia,"[60, 70)",2,0,0,0,2
9996,AfricanAmerican,Male,[60-70),?,Urgent,Discharged/transferred to SNF,Emergency Room,8,CM,Emergency/Trauma,...,False,"Acute, but ill-defined, cerebrovascular disease",Diabetes mellitus without mention of complicat...,Fitting and adjustment of other device,"[60, 70)",2,1,0,0,3
9997,AfricanAmerican,Female,[70-80),?,Emergency,Discharged/transferred to home with home healt...,Emergency Room,13,?,InternalMedicine,...,True,Streptococcal septicemia,Chronic kidney disease (ckd),Achlorhydria,"[70, 80)",3,0,2,0,5
9998,Caucasian,Male,[80-90),?,Urgent,Discharged/transferred to SNF,Emergency Room,2,MC,InternalMedicine,...,False,"Acute gastritis, without mention of hemorrhage",Hyperosmolality and/or hypernatremia,"Urinary tract infection, site not specified","[80, 90)",4,0,0,0,4


In [ ]:
import seaborn as sns

# make histogram of age variable split by readmitted
sns.displot(df, x='time_in_hospital', bins=30, kde=True,hue='readmitted')

In [ ]:
for int64_column in df.columns[df.dtypes == 'int64']:
    #print(int64_column)
    sns.displot(df, x=int64_column,hue='readmitted', stat='proportion', multiple='dodge')
    group_readmitted = df.loc[:,[int64_column,'readmitted']].groupby('readmitted')
    print(group_readmitted.describe())


In [ ]:
def group_and_value_count_by_readmitted(df,column):
    """Returns a dataframe with the count of each value in the specified column grouped by readmitted"""
    
    group_readmitted = df.loc[:,[column,'readmitted']].groupby('readmitted')

    return pd.concat([group_readmitted.value_counts(normalize=True),
               group_readmitted.value_counts()
               ],axis=1)

for cat_column in df.columns[df.dtypes == 'object']:
    print(group_and_value_count_by_readmitted(df,cat_column))

    

In [ ]:
medspec = group_and_value_count_by_readmitted(df,"medical_specialty")

# thoughts
- So - this ISNT an imbalanced dataset - so don't need to worry about that

In [ ]:
import sklearn

# use a tree model to predict readmission
from sklearn.tree import DecisionTreeClassifier

# import random forest classifier
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier

from sklearn.model_selection import train_test_split


# one-hot encode the categorical variables
df_onehot = pd.get_dummies(df.drop('readmitted',axis=1), drop_first=True)

# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df_onehot, 
                                                    df['readmitted'], test_size=0.30, 
                                                    random_state=101, stratify=df['readmitted'])

# create a decision tree classifier instance
model = GradientBoostingClassifier(n_estimators=100, max_depth=4)

# fit the decision tree classifier
model.fit(X_train,y_train)



train_predictions = model.predict(X_train)

# use the classifier to predict
predictions = model.predict(X_test)

# import the metrics class
from sklearn import metrics

# show the confusion matrix
print(metrics.confusion_matrix(y_test,predictions))

print(metrics.classification_report(y_train,train_predictions))

# show the accuracy, precision, and recall
print(metrics.classification_report(y_test,predictions))